In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time
import statistics

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
poses = mp_pose.Pose(
    static_image_mode=False,  #静止画かどうか
    # UPPER_BODY_ONLY=False,   #上半身のみかどうか
    # SMOOTH_LANDMARKS=True,   #ジッターを減らすかどうか
    min_detection_confidence=0.7, #検出成功の最低信頼値
    min_tracking_confidence=0.7 #追跡成功の最小信頼値
     ) 

target_vid =  cv2.VideoCapture("mv/fps10_afleg.mp4")
player_vid =  cv2.VideoCapture("mv/fps10_afleg.mp4")
score_path = "score_fp.csv"
# 59.94fps 1920p1080
if target_vid.get(cv2.CAP_PROP_FPS) != player_vid.get(cv2.CAP_PROP_FPS):
    print("It is not same FPS.")
    print("target video's FPS : "+ str(target_vid.get(cv2.CAP_PROP_FPS)))
    print("player video's FPS : "+ str(player_vid.get(cv2.CAP_PROP_FPS)))


In [3]:
# 座標をnumpyにする
def landmark2np(pose_landmarks):
    detected_point = 13
    li = []
    for j in (pose_landmarks.landmark):
            li.append([j.x, j.y, j.z])
    for k in li:
        if k[0] == 0 and k[1] == 0  and k[2]==0:
            print("No detected")
            li[k] = li[detected_point]

    return np.array(li) - li[detected_point]

# コサイン類似度を計算する

def manual_cos(A, B):
    dot = np.sum(A*B, axis=-1)
    A_norm = np.linalg.norm(A, axis=-1)
    B_norm = np.linalg.norm(B, axis=-1)
    cos = dot / (A_norm*B_norm+1e-10)

    for i in cos:
        count = 0
        if i == 0:
            print("cos deleted")
            np.delete(cos,count)
        count = count +1
    return cos.mean()

In [4]:
frames =  0
totalScore =  []

# try:
while True:
  target_success, target_img = target_vid.read()
  player_success, player_img = player_vid.read()
  if target_success == False:
    break
  if player_success == False:
    break
  targer_imgRGB = cv2.cvtColor(target_img, cv2.COLOR_BGR2RGB)
  player_imgRGB = cv2.cvtColor(player_img, cv2.COLOR_BGR2RGB)
  target_results = poses.process(targer_imgRGB)
  player_results = poses.process(player_imgRGB)

  if player_results.pose_landmarks:
      if target_results.pose_landmarks:
        target_mortion = landmark2np(target_results.pose_landmarks)
        player_mortion = landmark2np(player_results.pose_landmarks)
      
      score = manual_cos(target_mortion, player_mortion)
  if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  totalScore.append(score)
  frames = frames + 1
  print("SCORE : " + str(score))  
endScore = statistics.mean(totalScore)
print("AVE SCORE : " + str(endScore))
# except:
#     print("ERROR")

cos deleted
SCORE : 0.8963540494333083
cos deleted
SCORE : 0.9688844624935399
cos deleted
SCORE : 0.9695804385747376
cos deleted
SCORE : 0.9696736016085336
cos deleted
SCORE : 0.9695641576879248
cos deleted
SCORE : 0.9696113030832181
cos deleted
SCORE : 0.9696274700670837
cos deleted
SCORE : 0.9692154094099664
cos deleted
SCORE : 0.9683928395565656
cos deleted
SCORE : 0.9694828851343825
cos deleted
SCORE : 0.9677273700652844
cos deleted
SCORE : 0.9673094043081785
cos deleted
SCORE : 0.9686961286235821
cos deleted
SCORE : 0.9695009395008702
cos deleted
SCORE : 0.9695648271799193
cos deleted
SCORE : 0.9683052812970665
cos deleted
SCORE : 0.9644049467467768
cos deleted
SCORE : 0.9684997686605593
cos deleted
SCORE : 0.9691381278324037
cos deleted
SCORE : 0.9680025806858635
cos deleted
SCORE : 0.9690988306652623
cos deleted
SCORE : 0.96927280394636
cos deleted
SCORE : 0.969416430033734
cos deleted
SCORE : 0.9693650972513085
cos deleted
SCORE : 0.9695761774976308
cos deleted
SCORE : 0.969671

In [5]:
print(type(score))
np.savetxt(score_path,totalScore,delimiter=",")


<class 'numpy.float64'>
